In [1]:
import os
import torch
import numpy as np
import torchaudio
from datasets import load_dataset
from transformers import (
    Wav2Vec2FeatureExtractor,
    Wav2Vec2ForSequenceClassification,
    TrainingArguments,
    Trainer
)
from sklearn.metrics import accuracy_score, f1_score

In [2]:
AUDIO_DIR = "audio_clips"
METADATA_CSV = "clip_list - clip_list.csv"  # contains filename,label
OUTPUT_DIR = "wav2vec2_aggression_model"
MODEL_CHECKPOINT = "facebook/wav2vec2-base"
TARGET_SAMPLE_RATE = 16_000
MAX_DURATION_SEC = 5               # max length for padding/truncation
NUM_LABELS = 2                     # violent vs non-violent
LABEL2ID = {"Non Violent": 0, "Violent": 1}
ID2LABEL = {v: k for k, v in LABEL2ID.items()}

In [3]:
# 1. Load dataset from CSV
print("Loading metadata...")
ds = load_dataset(
    "csv",
    data_files=METADATA_CSV,
    split="train"
)

Loading metadata...


In [4]:
# 2. Rename clip_name to filename
print("Renaming columns...")
ds = ds.rename_column("clip_name", "filename")

Renaming columns...


In [5]:
print("Mapping labels...")
def map_labels(example):
    example["labels"] = LABEL2ID[example["class"]]
    return example

ds = ds.map(map_labels, remove_columns=["class"])

Mapping labels...


Map:   0%|          | 0/499 [00:00<?, ? examples/s]

In [6]:
ds = ds.train_test_split(test_size=0.2, seed=42)
train_ds = ds["train"]
eval_ds = ds["test"]

In [7]:
print("Loading model and feature extractor...")
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(MODEL_CHECKPOINT, return_attention_mask=True)
model = Wav2Vec2ForSequenceClassification.from_pretrained(
    MODEL_CHECKPOINT,
    num_labels=NUM_LABELS,
    problem_type="single_label_classification"
)

Loading model and feature extractor...


/Users/kaushalpatil/Development/USC MS CSAI Program/Multimodal Prob Learning /venv/lib/python3.10/site-packages/transformers/configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
def preprocess(batch):
    # Load waveform
    path = os.path.join(AUDIO_DIR, batch["filename"])
    waveform, sr = torchaudio.load(path)  # waveform shape: (channels, samples)
    # If necessary, resample to target sample rate
    if sr != TARGET_SAMPLE_RATE:
        resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=TARGET_SAMPLE_RATE)
        waveform = resampler(waveform)
        sr = TARGET_SAMPLE_RATE
    audio = waveform[0].numpy()

    # Feature extraction with padding/truncation
    inputs = feature_extractor(
        audio,
        sampling_rate=sr,
        padding="max_length",
        truncation=True,
        max_length=sr * MAX_DURATION_SEC,
        return_tensors="pt"
    )
    batch["input_values"] = inputs.input_values[0]
    batch["attention_mask"] = inputs.attention_mask[0]
    return batch

print("Preprocessing and training datasets...")
train_ds = train_ds.map(preprocess, remove_columns=["filename"])
eval_ds  = eval_ds.map(preprocess, remove_columns=["filename"])

Preprocessing and training datasets...


Map:   0%|          | 0/399 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [9]:
def compute_metrics(pred):
    preds = np.argmax(pred.predictions, axis=1)
    labels = pred.label_ids
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds)
    }


In [10]:
# 8. TrainingArguments & Trainer setup
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    logging_steps=50,
    eval_steps=100,
    save_steps=500,
    num_train_epochs=5,
    learning_rate=1e-5,
    do_train=True,
    fp16=torch.cuda.is_available(),
    do_eval=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)

/var/folders/xx/cq2k_b4j60l3mtp2k3x41s0w0000gn/T/ipykernel_25868/4128464920.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [11]:
print("Starting training...")
trainer.train()
os.makedirs(OUTPUT_DIR, exist_ok=True)
state_path = os.path.join(OUTPUT_DIR, "aggression_detector_state.pt")
torch.save(trainer.model.state_dict(), state_path)
full_model_path = os.path.join(OUTPUT_DIR, "aggression_detector_full.pt")
torch.save(trainer.model, full_model_path)
print(f"Saved state_dict to {state_path}")
print(f"Saved full model to {full_model_path}")

Starting training...


Step,Training Loss
50,0.618100
100,0.541000
150,0.473700
200,0.449300
250,0.424800


RuntimeError: Serialization of parametrized modules is only supported through state_dict(). See:
https://pytorch.org/tutorials/beginner/saving_loading_models.html#saving-loading-a-general-checkpoint-for-inference-and-or-resuming-training

In [12]:
metrics = trainer.evaluate()
print(f"Test Accuracy: {metrics['eval_accuracy']:.4f}")
print(f"Test F1 Score: {metrics['eval_f1']:.4f}")

Test Accuracy: 0.7800
Test F1 Score: 0.4500
